<a href="https://colab.research.google.com/github/seepala98/EVA-4/blob/master/PHASE_1/S3/Copy_of_EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1, bias=False) #input 28x28x1 Output 28x28x8
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1, bias=False) #input 28x28x8 Output 28x28x16
        self.bn2 = nn.BatchNorm2d(16)
        
        self.pool1 = nn.MaxPool2d(2, 2) #input 28x28x16 Output 14x14x16
        
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1, bias=False) #input 14x14x16 Output 14x14x32 
        self.bn3 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(32, 32, 3) #input 14x14x32 Output 12x12x32
        self.bn4 = nn.BatchNorm2d(32)

        self.pool2 = nn.MaxPool2d(2, 2) #input 12x12x32 Output 6x6x32
        
        self.conv5 = nn.Conv2d(32, 16, 1, bias=False) #input 6x6x32 Output 6x6x16
        self.bn5 = nn.BatchNorm2d(16)
        self.conv5_1 = nn.Conv2d(16, 16, 3, bias=False) #input 6x6x16 Output 4x4x16
        self.bn5_1 = nn.BatchNorm2d(16)
        
        self.gap6 = nn.AdaptiveAvgPool2d((1, 1)) #input 4x4x16 Output 1x1x16 
        self.conv7 = nn.Conv2d(16, 10, 1) #input 1x1x16 Output 1x1x10

    def forward(self, x):
        x = self.pool1(self.bn2(F.relu(self.conv2(self.bn1(F.relu(self.conv1(x)))))))
        x = self.pool2(self.bn4(F.relu(self.conv4(self.bn3(F.relu(self.conv3(x)))))))
        x = F.relu(self.gap6(self.bn5_1(F.relu(self.conv5_1(self.bn5(F.relu(self.conv5(x))))))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [11]:

!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 16, 28, 28]           1,152
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         MaxPool2d-5           [-1, 16, 14, 14]               0
            Conv2d-6           [-1, 32, 14, 14]           4,608
       BatchNorm2d-7           [-1, 32, 14, 14]              64
            Conv2d-8           [-1, 32, 12, 12]           9,248
       BatchNorm2d-9           [-1, 32, 12, 12]              64
        MaxPool2d-10             [-1, 32, 6, 6]               0
           Conv2d-11             [-1, 16, 6, 6]             512
      BatchNorm2d-12             [-1, 16, 6, 6]              32
           Conv2d-13             [-1, 16, 4, 4]           2,304
      BatchNorm2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'epoch = {epoch} loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [14]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum = 0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0432, Accuracy: 9869/10000 (98.6900%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0393, Accuracy: 9876/10000 (98.7600%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0258, Accuracy: 9925/10000 (99.2500%)



epoch = 19 loss=0.004013240337371826 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 33.01it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0230, Accuracy: 9943/10000 (99.4300%)



epoch = 20 loss=0.0003819465637207031 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 30.92it/s]



Test set: Average loss: 0.0211, Accuracy: 9947/10000 (99.4700%)

